In [8]:
import pandas as pd
from collections import Counter
import re

In [3]:
df =  pd.read_csv('tweet_all2.csv',  sep=';;', on_bad_lines='skip',lineterminator='\r', engine ='python')

print(len(df))

1170852


In [5]:
# Specify the list of specific words you want to filter out
specific_words = ['trânsito', 'rodovia', 'tráfego', 'ecovias','TIM', 'tim', 'Kaysar' , 'novela', 'Bandeirantes', 'operacaobetalab', 'bbb', 'transito', 'trânsito', 'orcars', 'oscar', 'grammy', 'grammys', 'kaysar', 'timbetalab', ' masterchefbr', 'enem', 'futebol', ' bundesliga', 'corinthians']

# Fill NaN values in the 'text' column with an empty string
df['text'].fillna('', inplace=True)

# Create a boolean mask for rows containing specific words in the 'text' column
mask = df['text'].str.contains('|'.join(specific_words), case=False)

# Apply the mask to filter out rows with specific words
df_filtered = df[~mask]

# Define a regular expression pattern to match URLs
url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

# Remove URLs from the 'text' column using regular expressions
df_filtered.loc[:, 'text'] = df_filtered['text'].str.replace(url_pattern, '', regex=True)

print("Filtered DataFrame length:", len(df_filtered))


Filtered DataFrame length: 1079401


C:\Users\anape\AppData\Local\Temp\ipykernel_2196\1100706827.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, 'text'] = df_filtered['text'].str.replace(url_pattern, '', regex=True)


In [6]:
df1 = df_filtered
df2 = pd.read_csv('hashtags.csv',  sep=';;', on_bad_lines='skip',lineterminator='\r', engine ='python')

# Convert 'id' column in df1 to string
df1['id'] = df1['id'].astype(str)

# Convert 'tweet' column in df2 to string
df2['tweet'] = df2['tweet'].astype(str)

# Fill NaN values in the 'tweet' column of df2
df2['tweet'].fillna('', inplace=True)

# Perform an inner join on 'id' and 'tweet' columns to include only rows with correspondence
merged_df = df1.merge(df2, left_on='id', right_on='tweet', how='inner')

# Print the length of the new merged dataframe
print("Filtered DataFrame length:", len(merged_df))

C:\Users\anape\AppData\Local\Temp\ipykernel_2196\3910538142.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['id'] = df1['id'].astype(str)


Filtered DataFrame length: 160756


In [7]:
# Calculate the frequency of values in the 'tag' column
tag_frequency = merged_df['tag'].value_counts()

# Print the frequency of tags from highest to lowest
print(tag_frequency)

G1                  6470
refugiados          5870
Venezuela           2172
EUA                 1820
Brasil              1735
                    ... 
valtellina_news        1
ESCREVEAI              1
Dekasegui              1
ViraVotoHaddad13       1
EsporteCultura         1
Name: tag, Length: 25460, dtype: int64


In [12]:
# Extract the 'year' and 'month' components from 'created_at'
merged_df['created_at'] = pd.to_datetime(merged_df['created_at'])  # Ensure 'created_at' is in datetime format
merged_df['year'] = merged_df['created_at'].dt.year
merged_df['month'] = merged_df['created_at'].dt.month

# Filter rows for the years 2015 to 2019
merged_df_interval = merged_df[(merged_df['year'] >= 2015) & (merged_df['year'] <= 2019)]

# Group the data by year, month, and tag
monthly_hashtags = merged_df_interval.groupby(['year', 'month'])['tag'].apply(lambda x: [tag.lower() for tag in x])

# Count the occurrences of each tag per month
monthly_tag_counts = monthly_hashtags.apply(Counter)

# Get the top 100 tags by month in a list format
top_tags_by_month = []
for (year, month), tag_counts in monthly_tag_counts.items():
    top_tags = tag_counts.most_common(100)
    top_tags_by_month.append((f"{year}-{month:02}", top_tags))

# Print the top 30 tags for each month in list format
for year_month, tags in top_tags_by_month:
    print(f"Top 100 tags for {year_month}:")
    for tag, count in tags:
        print(f"{tag}: {count}")
    print()

Top 100 tags for 2015.0-2.0:
cpbr8: 133
tvonline: 72
seguesigodevolta: 38
g1: 36
opcoiote: 36
rt1: 35
refugiados: 34
betacaralhudosan: 33
notícias: 25
msf: 24
reciclagem: 24
andremartinellinamash: 21
eumigrante: 20
cuecasmash: 20
sp: 18
dialogosconectas: 17
mundo: 16
rgp: 16
uol: 15
gratuita: 15
atençãonasestradas: 14
personalizada1: 13
campusparty: 12
news: 12
feedbrasil: 12
onu: 12
agenda: 12
acnur: 11
angola: 11
migrante: 11
imigração: 10
spfc: 10
palestina: 10
siria: 10
binacional: 10
michoacan: 10
imigrantes: 9
internacional: 9
opentosyria: 9
fórum: 9
spidernocombate: 7
brasil: 7
pt: 7
teamfollowback: 7
sigodevolta: 7
vídeosp: 7
museu: 7
ceidpu: 6
daleplay: 6
barlovento: 6
feedly: 6
atendimento_pré_hospitalar: 6
papa: 6
tv: 6
síria: 6
betalab: 6
ny: 6
saopaulo: 5
minuto: 5
folha: 5
eua: 5
mujica: 5
uruguai: 5
rt: 5
agora: 5
maismédicos: 5
ucrânia: 5
intercâmbio: 5
br: 5
spglobal: 5
japão: 4
sampa: 4
r7: 4
asiloparahassanna: 4
dayumbetauhu: 4
edgar: 4
arturo: 4
askdemigod: 4
globo:

In [14]:
# Get the top 30 tags by month in a list format
top_tags_by_month = []
for (year, month), tag_counts in monthly_tag_counts.items():
    top_tags = tag_counts.most_common(100)
    top_tags_by_month.append((f"{year}-{month:02}", top_tags))

# Print the top tags for each month in flat list format
for year_month, tags in top_tags_by_month:
    print(f"Top tags for {year_month}:")
    for tag, count in tags:
        print(f"{tag} ({count}), ", end="")
    print()

Top tags for 2015.0-2.0:
cpbr8 (133), tvonline (72), seguesigodevolta (38), g1 (36), opcoiote (36), rt1 (35), refugiados (34), betacaralhudosan (33), notícias (25), msf (24), reciclagem (24), andremartinellinamash (21), eumigrante (20), cuecasmash (20), sp (18), dialogosconectas (17), mundo (16), rgp (16), uol (15), gratuita (15), atençãonasestradas (14), personalizada1 (13), campusparty (12), news (12), feedbrasil (12), onu (12), agenda (12), acnur (11), angola (11), migrante (11), imigração (10), spfc (10), palestina (10), siria (10), binacional (10), michoacan (10), imigrantes (9), internacional (9), opentosyria (9), fórum (9), spidernocombate (7), brasil (7), pt (7), teamfollowback (7), sigodevolta (7), vídeosp (7), museu (7), ceidpu (6), daleplay (6), barlovento (6), feedly (6), atendimento_pré_hospitalar (6), papa (6), tv (6), síria (6), betalab (6), ny (6), saopaulo (5), minuto (5), folha (5), eua (5), mujica (5), uruguai (5), rt (5), agora (5), maismédicos (5), ucrânia (5), int